# Data Preprocessing

### Remove empty columns/rows, merge datasets into one dataframe for training and testing 
### match up countries 

In [1]:
pip install pandas beautifulsoup4 requests

Note: you may need to restart the kernel to use updated packages.


In [78]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pyspark.sql import SparkSession

In [79]:
spark= SparkSession.builder \
        .appName("Pyspark SQL in Jupyter") \
        .getOrCreate()

24/08/15 16:59:43 WARN Utils: Your hostname, Kevins-MacBook-Pro-97.local resolves to a loopback address: 127.0.0.1; using 10.1.10.100 instead (on interface en0)
24/08/15 16:59:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/15 16:59:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Scraping

## Makes a request at the PRIS IAEA URL, parses html to find and read table, to finally turn it into a csv

In [3]:
url= "https://pris.iaea.org/PRIS/WorldStatistics/OperationalReactorsByCountry.aspx"
response=requests.get(url)
soup=BeautifulSoup(response.text, 'html.parser')

table=soup.find('table')

df=pd.read_html(str(table))[0]

df.to_csv('operational_reactors_by_country.csv_2024', index=False)

# Net Electrical Capacity (MW) and Number of Reactors by Country

In [82]:
country_electrical_capacity = spark.read.csv('operational_reactors_by_country_2024.csv', header=True, inferSchema=True)
country_electrical_capacity.show()

+--------------------+-----------------------------------+------------------+----------+
|             Country|Total Net Electrical Capacity  [MW]|Number of Reactors|Unnamed: 3|
+--------------------+-----------------------------------+------------------+----------+
|           ARGENTINA|                               1641|                 3|      null|
|             ARMENIA|                                416|                 1|      null|
|             BELARUS|                               2220|                 2|      null|
|             BELGIUM|                               3908|                 5|      null|
|              BRAZIL|                               1884|                 2|      null|
|            BULGARIA|                               2006|                 2|      null|
|              CANADA|                              13699|                19|      null|
|               CHINA|                              54152|                56|      null|
|      CZECH REPUBLIC

## Remove problematic column using SQL

In [93]:
country_electrical_capacity.createOrReplaceTempView("my_table")


# SQL query 
results= spark.sql("ALTER TABLE my_table DROP COLUMN `Unnamed 3`;")
# country_electrical_capacity_clean = country_electrical_capacity.drop('Unnamed: 3', axis=1)
# country_electrical_capacity_clean

AnalysisException: my_table is a temp view. 'ALTER TABLE ... DROP COLUMNS' expects a table.; line 1 pos 12

# Lithium dataset
## MCS 2024 Provides information on Country, extraction type, true production in 2022, estimated production in 2023, production notes, reserves in tons, and reserves notes
### The Mineral Commodities Summary withholds information on the United States to avoid disclosing company proprietary data 

In [21]:
lithium_2024 = pd.read_csv('mcs2024-lithi_world.csv')
lithium_2024

,Source,Country,Type,Prod_t_2022,Prod_t_est_2023,Prod_notes,Reserves_t,Reserves_notes,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,MCS2024,United States,"Mine production, lithium content",W,W,NaN,1100000.0,"Reserves for Argentina, Australia, Brazil, Chi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MCS2024,Argentina,"Mine production, lithium content",6590,9600,NaN,3600000.0,"Reserves for Argentina, Australia, Brazil, Chi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MCS2024,Australia,"Mine production, lithium content",74700,86000,NaN,6200000.0,"For Australia, Joint Ore Reserves Committee-co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MCS2024,Brazil,"Mine production, lithium content",2630,4900,Also estimated in 2022,390000.0,"Reserves for Argentina, Australia, Brazil, Chi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MCS2024,Canada,"Mine production, lithium content",520,3400,Also estimated in 2022,930000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MCS2024,Chile,"Mine production, lithium content",38000,44000,NaN,9300000.0,"Reserves for Argentina, Australia, Brazil, Chi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,MCS2024,China,"Mine production, lithium content",22600,33000,Also estimated in 2022,3000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MCS2024,Portugal,"Mine production, lithium content",380,380,Also estimated in 2022,60000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MCS2024,Zimbabwe,"Mine production, lithium content",1030,3400,Also estimated in 2022,310000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,MCS2024,Other countries6,"Mine production, lithium content",0,0,NaN,2800000.0,"Reserves for Argentina, Australia, Brazil, Chi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Remove empty rows and columns

### Remove empty columns

In [57]:
# if the index of the column is greater than the index of 'Reserves_notes' (the last unproblematic column) then we add it to the remove list
# columns.values is just the names of each of the columns, then we get the index based on the name as the key using columns.get_loc
# then we compare indices
# finally drop the remove list with axis=1 (columns)

remove_cols = [i for i in lithium_2024.columns.values if lithium_2024.columns.get_loc(i)  > lithium_2024.columns.get_loc('Reserves_notes')]

lithium_2024_no_bad_cols = lithium_2024.drop(remove_cols, axis=1)
lithium_2024_no_bad_cols

,Source,Country,Type,Prod_t_2022,Prod_t_est_2023,Prod_notes,Reserves_t,Reserves_notes
0,MCS2024,United States,"Mine production, lithium content",W,W,NaN,1100000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
1,MCS2024,Argentina,"Mine production, lithium content",6590,9600,NaN,3600000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
2,MCS2024,Australia,"Mine production, lithium content",74700,86000,NaN,6200000.0,"For Australia, Joint Ore Reserves Committee-co..."
3,MCS2024,Brazil,"Mine production, lithium content",2630,4900,Also estimated in 2022,390000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
4,MCS2024,Canada,"Mine production, lithium content",520,3400,Also estimated in 2022,930000.0,NaN
5,MCS2024,Chile,"Mine production, lithium content",38000,44000,NaN,9300000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
6,MCS2024,China,"Mine production, lithium content",22600,33000,Also estimated in 2022,3000000.0,NaN
7,MCS2024,Portugal,"Mine production, lithium content",380,380,Also estimated in 2022,60000.0,NaN
8,MCS2024,Zimbabwe,"Mine production, lithium content",1030,3400,Also estimated in 2022,310000.0,NaN
9,MCS2024,Other countries6,"Mine production, lithium content",0,0,NaN,2800000.0,"Reserves for Argentina, Australia, Brazil, Chi..."


### Remove empty rows

In [72]:
# same procedure; problematic rows start after row 10, so if row number is greater than 10, we will remove 

remove_rows = (i for i in range(11, lithium_2024_no_bad_cols.shape[0]))
# .shape gives the dimensions of the data in format (rows, columns) 
# example: lithium_2024_no_bad_colss has shape (26,8) 26 rows, 8 columns

lithium_2024_no_bad_cols_rows = lithium_2024_no_bad_cols.drop(remove_rows, axis=0)
lithium_2024_no_bad_cols_rows

,Source,Country,Type,Prod_t_2022,Prod_t_est_2023,Prod_notes,Reserves_t,Reserves_notes
0,MCS2024,United States,"Mine production, lithium content",W,W,NaN,1100000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
1,MCS2024,Argentina,"Mine production, lithium content",6590,9600,NaN,3600000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
2,MCS2024,Australia,"Mine production, lithium content",74700,86000,NaN,6200000.0,"For Australia, Joint Ore Reserves Committee-co..."
3,MCS2024,Brazil,"Mine production, lithium content",2630,4900,Also estimated in 2022,390000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
4,MCS2024,Canada,"Mine production, lithium content",520,3400,Also estimated in 2022,930000.0,NaN
5,MCS2024,Chile,"Mine production, lithium content",38000,44000,NaN,9300000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
6,MCS2024,China,"Mine production, lithium content",22600,33000,Also estimated in 2022,3000000.0,NaN
7,MCS2024,Portugal,"Mine production, lithium content",380,380,Also estimated in 2022,60000.0,NaN
8,MCS2024,Zimbabwe,"Mine production, lithium content",1030,3400,Also estimated in 2022,310000.0,NaN
9,MCS2024,Other countries6,"Mine production, lithium content",0,0,NaN,2800000.0,"Reserves for Argentina, Australia, Brazil, Chi..."


## Remove columns not useful to the analysis

### For the purpose of our analysis, we don't need columns: Source, Type, Prod Notes, Reserves Notes

In [73]:
remove_unnecessary = ['Source',
                      'Type',
                      'Prod_notes',
                      'Reserves_notes'
                     ]
lithium_2024_clean = lithium_2024_no_bad_cols_rows.drop(remove_unnecessary, axis=1)
lithium_2024_clean

,Country,Prod_t_2022,Prod_t_est_2023,Reserves_t
0,United States,W,W,1100000.0
1,Argentina,6590,9600,3600000.0
2,Australia,74700,86000,6200000.0
3,Brazil,2630,4900,390000.0
4,Canada,520,3400,930000.0
5,Chile,38000,44000,9300000.0
6,China,22600,33000,3000000.0
7,Portugal,380,380,60000.0
8,Zimbabwe,1030,3400,310000.0
9,Other countries6,0,0,2800000.0


# Rare Earth Ores dataset

# Since its from the mcs as well, provides info in the same format as the Lithium dataset
### All preprocessing will be done in one block

In [75]:
rareearth_2024=pd.read_csv('mcs2024-raree_world.csv')
rareearth_2024

,Source,Country,Type,Prod_t_est_2022,Prod_t_est_2023,Prod_notes,Reserves_t,Reserves_notes,Unnamed: 8,Unnamed: 9,...,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43
0,MCS2024,United States,"Rare earths, mine production, rare-earth-oxide...",42000.0,43000.0,NaN,1800000.0,"Reserves for Australia, Russia, Thailand, and ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MCS2024,Australia,"Rare earths, mine production, rare-earth-oxide...",18000.0,18000.0,NaN,5700000.0,"For Australia, Joint Ore Reserves Committee-co...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MCS2024,Brazil,"Rare earths, mine production, rare-earth-oxide...",80.0,80.0,NaN,21000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MCS2024,Burma,"Rare earths, mine production, rare-earth-oxide...",12000.0,38000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MCS2024,Canada,"Rare earths, mine production, rare-earth-oxide...",0.0,0.0,NaN,830000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MCS2024,China,"Rare earths, mine production, rare-earth-oxide...",210000.0,240000.0,Production quota; does not include undocumente...,44000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,MCS2024,Greenland,"Rare earths, mine production, rare-earth-oxide...",0.0,0.0,NaN,1500000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MCS2024,India,"Rare earths, mine production, rare-earth-oxide...",2900.0,2900.0,NaN,6900000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MCS2024,Madagascar,"Rare earths, mine production, rare-earth-oxide...",960.0,960.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,MCS2024,Malaysia,"Rare earths, mine production, rare-earth-oxide...",80.0,80.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
remove_cols = [i for i in rareearth_2024.columns.values if rareearth_2024.columns.get_loc(i)  > rareearth_2024.columns.get_loc('Reserves_notes')]

rareearth_2024_no_bad_cols = rareearth_2024.drop(remove_cols, axis=1)

remove_rows = (i for i in range(16, rareearth_2024_no_bad_cols.shape[0]))

rareearth_2024_no_bad_cols_rows = rareearth_2024_no_bad_cols.drop(remove_rows, axis=0)

remove_unnecessary = ['Source',
                      'Type',
                      'Prod_notes',
                      'Reserves_notes'
                     ]

rareearth_2024_clean = rareearth_2024_no_bad_cols_rows.drop(remove_unnecessary, axis=1)
rareearth_2024_clean

,Country,Prod_t_est_2022,Prod_t_est_2023,Reserves_t
0,United States,42000.0,43000.0,1800000.0
1,Australia,18000.0,18000.0,5700000.0
2,Brazil,80.0,80.0,21000000.0
3,Burma,12000.0,38000.0,NaN
4,Canada,0.0,0.0,830000.0
5,China,210000.0,240000.0,44000000.0
6,Greenland,0.0,0.0,1500000.0
7,India,2900.0,2900.0,6900000.0
8,Madagascar,960.0,960.0,NaN
9,Malaysia,80.0,80.0,NaN


# Zirconium and Hafnium dataset

In [9]:
zirco_2024=pd.read_csv('mcs2024-zirco_world.csv')
zirco_2024

,Source,Country,Type,Prod_kt_est_2022,Prod_kt_est_2023,Prod_notes,Reserves_kt_2023,Reserves_notes
0,MCS2024,United States,"Zirconium ores and zircon concentrates, mine p...",100,100,Data are rounded to the nearest hundred thousa...,500.0,NaN
1,MCS2024,Australia,"Zirconium ores and zircon concentrates, mine p...",500,500,NaN,55000.0,"For Australia, Joint Ore Reserves Committee-co..."
2,MCS2024,China,"Zirconium ores and zircon concentrates, mine p...",140,140,NaN,72.0,"Zirconium reserves for Australia, China, Mozam..."
3,MCS2024,Indonesia,"Zirconium ores and zircon concentrates, mine p...",97,90,NaN,NaN,NaN
4,MCS2024,Kenya,"Zirconium ores and zircon concentrates, mine p...",27,30,NaN,18.0,NaN
5,MCS2024,Madagascar,"Zirconium ores and zircon concentrates, mine p...",27,30,NaN,2300.0,NaN
6,MCS2024,Mozambique,"Zirconium ores and zircon concentrates, mine p...",104,90,NaN,1500.0,"Zirconium reserves for Australia, China, Mozam..."
7,MCS2024,Senegal,"Zirconium ores and zircon concentrates, mine p...",57,50,NaN,2600.0,NaN
8,MCS2024,Sierra Leone,"Zirconium ores and zircon concentrates, mine p...",34,30,NaN,290.0,NaN
9,MCS2024,South Africa,"Zirconium ores and zircon concentrates, mine p...",300,400,NaN,5600.0,"Zirconium reserves for Australia, China, Mozam..."


# Since its from the mcs as well, provides info in the same format as the Lithium dataset
### All preprocessing will be done in one block

In [77]:
remove_unnecessary = ['Source',
                      'Type',
                      'Prod_notes',
                      'Reserves_notes'
                     ]

zirco_2024_clean = zirco_2024.drop(remove_unnecessary, axis=1)
zirco_2024_clean

,Country,Prod_kt_est_2022,Prod_kt_est_2023,Reserves_kt_2023
0,United States,100,100,500.0
1,Australia,500,500,55000.0
2,China,140,140,72.0
3,Indonesia,97,90,NaN
4,Kenya,27,30,18.0
5,Madagascar,27,30,2300.0
6,Mozambique,104,90,1500.0
7,Senegal,57,50,2600.0
8,Sierra Leone,34,30,290.0
9,South Africa,300,400,5600.0


In [74]:

# lithium_sali_2024=pd.read_csv('mcs2024-lithi_salient.csv') ? 
# thorium_sali_2024 = pd.read_csv('mcs2024-thori_salient.csv') ?


ocean_2024=pd.read_csv('cleaned_wod_data.csv') # fix file 